# Widefield analysis; point-spread-function

This code walks you through the following. 'A-C' generalize to any widefield experiment.  'D' is specific to a retinotopy experiment.  On each trial a bar is flashed on the screen at a different position.  If the bar is horizontal, it varies along the vertical axis and spans the entire width of the screen. If the bar is vertical, it varies along the horizontal axis and spans the entire height of the screen. This allows us to calculate the peak-location and width of the receptive field at each pixel.  This information is then used t

A) Loading an experiment. Two objects get built:
1. "analyzer class" -  built from the .analyzer file (metadata) of the given experiment, which contains all the visual stimulus information and image acquisition information (e.g. time stamps relative to stimuli).  The analyzer class is used in all data acquisition methods, such as two-photon imaging.  
2. "wf_analyzer class" - has attributes and functions relevant to wide-field analyses specifically. 

B) Getting the stimulus-evoked average response of each stimulus condition, normalized by a baseline.
C) Averaging the response within a specified time window.

D) Calculate the retinotopic map.

E) Calculate the receptive field width of each pixel.

F) Use 'D' and 'E' to calculate the point-spread function in the cortex.



# Load modules and classes

In [ ]:
"""
@author: IN
"""

import os
import matplotlib.pyplot as plt
import numpy as np


from analyzer_mod import analyzer #analyzer class
from wf_analysis_mod import wf_analyzer  #wf_analyzer class

# Pick an animal and experiment to analyze. Identify root location of .analyzer file (metadata) and images (data).

anim - animal name

expt - experiment name

ISIflag - intrinsic signal imaging (=1); fluorescence GCaMP imaging (=0).  This sets some of the pre-processing parameters. Unfortunately, this imaging modality information is not something that is reliably identified within the metadata files, but only in the "notes".  It could also easily be determined automatically by the statistics of the raw images.

In [ ]:

#anim = 'xt7'
#expt = 'u009_004'; #epi ret expt
#ISIflag = 1 

#anim = 'xt8'
#expt = 'u003_009' 
#ISIflag = 1
#pixpermm = 79 

anim = 'yd3'
expt = 'u001_002'
pixpermm = 79
ISIflag = 0

rootanadir = '/Volumes/TOSHIBA EXT/AnalyzerFiles/' #location of analyzer files
rootdatadir = '/Volumes/TOSHIBA EXT/ISIData/'      #location of data (images)

# Create the analyzer object from the .analyzer file. 
i.e. get the metadata. This contains all the information in the GUIs at the experiment terminal. Each attribute is a dictionary:

analyzer.M  #Dict of info in the 'MW' GUI.  e.g. analyzer.M['screen_dist'].  
analyzer.L  #Dict of info in the 'Looper' GUI.   
analyzer.P  #Dict of info in the 'ParamList' GUI. 
analyzer.looper  #Detailed info of how the trials were played out, for a given random seed.  
analyzer.ACQ  #Dict of info in the 'imager' GUI.  Contains the acquisition info, such as image size and acq rate.

In [ ]:
anafname = anim + '_' + expt + '.analyzer'
anapath = os.path.join(*[rootanadir, anim , anafname])

analyzerX = analyzer(anapath) 

# Create the wf_analysis object.  
This mostly consists of useful methods for analyzing widefield data.  e.g. wf_analysis.loaddata()

In [ ]:
#Create the widefield analysis object. Uses data file.

datapath = os.path.join(*[rootdatadir,anim,expt])  #Just the folder. Each frame is a separate file.

wfX = wf_analyzer(datapath,analyzerX)  

# Set trial averaging window
This sets some reasonable values for the given imaging method and trial length.  e.g. ISI is much slower, so the averaging window probably shouldn't start until about 1 sec after stimulus onset.

'baselineWin' - The time window (ms) for getting the average image prior to the stimulus-evoked activity.  Zeros is referenced to the beginning of stimulus onset. Positive is after the onset.

'responseWin' - The time window (ms) for getting the average image when we expect there to be stimulus-evoked activity.  Zeros is referenced to the beginning of stimulus onset. Positive is after the onset.

In [ ]:
if ISIflag:
    baselineWin = [-500, 500] #ms [start, stop]
    responseWin = [1000,  analyzerX.P['stim_time']*1000+2000]


else:   #GCaMP
    baselineWin = [-500, 0] #ms [start, stop]
    responseWin = [100, analyzerX.P['stim_time']*1000+500]

# Get a baseline and response image for each trial

In [ ]:

RMat = np.zeros((wfX.imsize[0],wfX.imsize[1],analyzerX.ntrials));
BMat = np.zeros((wfX.imsize[0],wfX.imsize[1],analyzerX.ntrials));

for tno in range(analyzerX.ntrials):  #loop trials
    
    print(tno)
    Rxt = wfX.loadtrial(tno,responseWin,analyzerX)
    Bxt = wfX.loadtrial(tno,baselineWin,analyzerX)
    
    if ISIflag:              #Invert, b/c ISI is a negative signal.
        bitdepth = 16;  #Pco Panda
        Rxt = 2^bitdepth - Rxt;
  
        
    RMat[:,:,tno] = np.mean(Rxt,axis = 2); #Store the mean response image of this trial
    BMat[:,:,tno] = np.mean(Bxt,axis = 2); #Store the mean baseline image of this trial


# Baseline normalization
Usually, we compute the dF/F on every trial.

In [ ]:
#No normalization
#RMat2 = RMat   

#Use the mean baseline across all trials in the denominator.  This can be more stable.
#RMat2 = (RMat-BMat) + mean(BMat,3)  

#Compute the dF/F on every trial
RMat2 = (RMat-BMat)/BMat 

In [ ]:
# Organize the images by condition number and take mean over the repeats.  
# We are left with one image from each condition after this

RMat3 = np.zeros((wfX.imsize[0],wfX.imsize[1],analyzerX.nconditions))
BMat3 = np.zeros((wfX.imsize[0],wfX.imsize[1],analyzerX.nconditions))

for c in range(analyzerX.nconditions):
    
    nr = analyzerX.nrepeats(c);
  
    tnos = np.zeros(nr).astype(int)
    for r in range(nr):
        tnos[r] = analyzerX.trialno(c,r)

    RMat3[:,:,c] = np.mean(RMat2[:,:,tnos],axis = 2)  #mean over repeats
    BMat3[:,:,c] = np.mean(BMat[:,:,tnos],axis = 2)
    
    
blankflag = 0
if analyzerX.loops['conds'][-1]['symbol'][0] == 'blank':
    blankflag = 1

In [ ]:
# All the code up until this point could be applied to any experiment.  
# Here is where it starts to assume certain looper variable 

#get the index within the looper
for p in range(len(analyzerX.L['param'])):
    if analyzerX.L['param'][p][0] == 's_phase2':
        phaseDim = p

    if analyzerX.L['param'][p][0] == 'ori2':
        oriDim = p

#valmat is a matrix containing the looper values for each condition

valMat = np.zeros((len(analyzerX.loops['conds'][c]['val']),analyzerX.nconditions-blankflag))
                  
for c in range(analyzerX.nconditions-blankflag):
    for p in range(len(analyzerX.loops['conds'][c]['val'])):
        valMat[p,c] = analyzerX.loops['conds'][c]['val'][p]
        


oridom = list(set(valMat[oriDim,:]))
phasedom = list(set(valMat[phaseDim,:]))


In [ ]:
# Plot mean response to each condition


plt.figure()
k = 1
for ori in range(len(oridom)):
    for p in range(len(phasedom)):

        a = np.where(valMat[oriDim,:] == oridom[ori])[0]
        b = np.where(valMat[phaseDim,:] == phasedom[p])[0]
        id = np.intersect1d(a,b)[0]
        
        imdum = RMat3[:,:,id];
        
        plt.subplot(len(oridom),len(phasedom),k)
  
        
        plt.imshow(imdum*bw)
        
        k += 1